<a href="https://colab.research.google.com/github/Blutronics/OffensiveTwitterDetection/blob/main/OffensiveDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

In [41]:
data=pd.read_csv("twitter.csv")
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [42]:
classdf=data[['class','tweet']]
classdf.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [43]:
tweets=classdf.tweet.to_string()

In [44]:
tweets = re.sub("[^A-Za-z" "]+"," ", tweets).lower()
tweets = re.sub("[0-9" "]+"," ", tweets)

In [45]:
class2df=classdf.loc[classdf['class']==2]
class2df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
40,2,""" momma said no pussy cats inside my doghouse """
63,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://..."
66,2,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo..."
67,2,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these..."


In [46]:
offensivedf=classdf.loc[classdf['class']==1]
offensivedf.head()

,class,tweet
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."


In [47]:
hatedf=classdf.loc[classdf['class']==0]
hatedf.head()

,class,tweet
85,0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1..."
89,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ..."
110,0,"""@DevilGrimz: @VigxRArts you're fucking gay, b..."
184,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL..."
202,0,"""@NoChillPaz: ""At least I'm not a nigger"" http..."


In [48]:
offensivetweets=offensivedf.tweet.to_string()


In [49]:
offensivetweets = re.sub("[^A-Za-z" "]+"," ", offensivetweets).lower()
offensivetweets = re.sub("[0-9" "]+"," ", offensivetweets)

In [50]:
offensivetweets

' rt mleew boy dats cold tyga dwn ba rt urkindofbrand dawg rt sbaby rt c g anderson viva based she lo rt shenikaroberts the shit you t madison x the shit just brighterdays i can not just sit up selfiequeenbri cause i m tired of amp you might not get ya bitch back amp rhythmixx hobbies include fighting maria keeks is a bitch she curves everyone lol i murda gang bitch its gang land so hoes that smoke are losers yea go bad bitches is the only thing that i like bitch get up off me bitch nigga miss me with it bitch plz whatever bitch who do you love bitches get cut off everyday b black bottle amp a bad bitch broke bitch cant tell me nothing cancel that bitch like nino cant you see these hoes wont change fuck no that bitch dont even suck dick got ya bitch tip toeing on my hardwood floor her pussy lips like heaven doors hoe what its hitting for i met that pussy on ocean dr i gave that p i need a trippy bitch who fuck on hennessy i spend my money how i want bitch its my bus i txt my old bitch 

In [51]:
offensivetweets = offensivetweets.split(" ")

In [52]:
hatespeachtweets=hatedf.tweet.to_string()


In [53]:
hatespeachtweets = re.sub("[^A-Za-z" "]+"," ", hatespeachtweets).lower()
hatespeachtweets = re.sub("[0-9" "]+"," ", hatespeachtweets)

In [54]:
hatespeachtweets

' blackman tide whalelookyhere howdydowdy cb baby white thunduh alsarabsss hes a devilgrimz vigxrarts you re fucking gay b markroundtreejr lmfaoooo i hate black peopl nochillpaz at least i m not a nigger http notoriousbm whiteponyjr ariza is a sna rtnba drakes new shoes that will be release theomaxximus gerryshalloweenparty http t ashlingwilde itsnotadam is bored supposed bigbootybishopp white thunduh lassen cc jayswaggkillah jackies a retard blondeprob jgabsss stacey dash won http t don t worry about the nigga you see worry ab hey go look at that video of the man that fou let s kill cracker babies wtf did i just h my grandma used to call me a porch monkey all nah its you nomeek justmilz our people now is the time for the aryan ra these sour apple bitter bitches i m not fuck we hate niggers we hate faggots and we hate we re out here and we re queer n who the fuck you callin jiggaboo nigga why people think gay marriage is okay is beyo you ain t gunna do shit spear chucker you ol trout m

In [55]:
hatespeachtweets = hatespeachtweets.split(" ")

In [56]:
#hatespeachtweets

In [57]:
vectorizer = TfidfVectorizer()
offensive = vectorizer.fit_transform(offensivetweets)

In [58]:
with open("stop.txt","r") as sw:
    stop_words = sw.read()

In [59]:
stop_words = stop_words.split("\n")

In [60]:
stop_words.extend(["rt","a"])

In [61]:
offensivetweets = [w for w in offensivetweets if not w in stop_words]


In [62]:
offensivetweets = " ".join(offensivetweets)

In [63]:
'''
wordcloud_offensive = WordCloud(
                      background_color='White',
                      width=1800,
                      height=1400
                     ).generate(offensivetweets)'''

"\nwordcloud_offensive = WordCloud(\n                      background_color='White',\n                      width=1800,\n                      height=1400\n                     ).generate(offensivetweets)"

In [64]:
#plt.imshow(wordcloud_offensive)

In [65]:
offensivetweets = offensivetweets.split(" ")

In [66]:
letter_counts = Counter(offensivetweets)
letter_counts.pop('')


1

In [67]:
most_offensive = letter_counts.most_common(15)
most_offensive

[('bitch', 4601),
 ('bitches', 1784),
 ('hoes', 1377),
 ('pussy', 1059),
 ('hoe', 1027),
 ('don', 714),
 ('fuck', 696),
 ('ass', 619),
 ('nigga', 524),
 ('ain', 505),
 ('shit', 440),
 ('niggas', 418),
 ('lol', 407),
 ('yo', 406),
 ('love', 364)]

In [68]:
hatespeachtweets = [w for w in hatespeachtweets if not w in stop_words]

In [69]:
letter_counts2 = Counter(hatespeachtweets)
letter_counts2.pop('')

2

In [70]:
most_hate = letter_counts2.most_common(15)
most_hate

[('faggot', 102),
 ('bitch', 98),
 ('white', 75),
 ('fuck', 73),
 ('nigga', 60),
 ('ass', 54),
 ('trash', 54),
 ('fag', 51),
 ('niggas', 50),
 ('nigger', 48),
 ('fucking', 46),
 ('hate', 43),
 ('don', 39),
 ('people', 34),
 ('faggots', 31)]